## Dataset Source : https://www.kaggle.com/puneet6060/intel-image-classification

## Importing Tensorflow

In [ ]:
import tensorflow as tf

## Making Flow for getting image

In [ ]:
TRAINING_DIR = './archive/train/'
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1 / 255)
train_generator = train_datagen.flow_from_directory(
    TRAINING_DIR,
    batch_size=128,
    classes=['building', 'forest', 'glacier', 'mountain', 'sea', 'street'],
    color_mode='rgb',
    class_mode='sparse',
    target_size=(150, 150)
)

VALIDATION_DIR = './archive/test/'
validation_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale= 1/255)
validation_generator = validation_datagen.flow_from_directory(
    VALIDATION_DIR,
    batch_size=128,
    classes=['building', 'forest', 'glacier', 'mountain', 'sea', 'street'],
    color_mode='rgb',
    class_mode='sparse',
    target_size=(150, 150)
)

## Making CNN Model

In [ ]:
def my_model():
    input_ = tf.keras.layers.Input((150, 150, 3))
    
    net = tf.keras.layers.Conv2D(16, (3, 3), strides=(1, 1), padding='valid', activation='relu')(input_)
    net = tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2))(net)
    net = tf.keras.layers.Dropout(0.2)(net)
    
    net = tf.keras.layers.Conv2D(32, (3, 3), strides=(1, 1), padding='same', activation='relu')(net)
    net = tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2))(net)
    net = tf.keras.layers.Dropout(0.2)(net)
    
    net = tf.keras.layers.Flatten()(net)
    out = tf.keras.layers.Dense(6, activation='softmax')(net)
    
    model = tf.keras.Model(inputs=[input_], outputs=[out])

    model.compile(loss=tf.keras.losses.sparse_categorical_crossentropy, 
                  optimizer=tf.keras.optimizers.Adam(lr=0.001), 
                  metrics=['accuracy'])
    model.summary()
    tf.keras.utils.plot_model(model, to_file='./model.png', show_shapes=True)
    return model

In [ ]:
model = my_model()

## Making Callbacks

In [ ]:
# Define a Callback class that stops training once accuracy reaches 99.9%
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('val_accuracy')>0.95):
      print("\nReached 95% accuracy so cancelling training!")
      self.model.stop_training = True
        
reduceLROnPlat = tf.keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.95,
        patience=2,
        verbose=1,
        mode='min',
        min_delta=0.0001,
        cooldown=2,
        min_lr=1e-5
)

checkpoint = tf.keras.callbacks.ModelCheckpoint(
    "./model_best.h5", monitor='val_loss', verbose=1, save_best_only=True,
    save_weights_only=False, mode='auto', save_freq='epoch'
)

my_callback = myCallback()

## Fitting model with data

In [ ]:
history = model.fit_generator(
    train_generator,
    epochs=100,
    verbose=1,
    validation_data=validation_generator,
    callbacks=[reduceLROnPlat, my_callback, checkpoint]
)

## Visualization

In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='validation')
plt.title('Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

In [ ]:
plt.plot(history.history['accuracy'], label='train')
plt.plot(history.history['val_accuracy'], label='validation')
plt.title('Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()